In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import os.path as op
import glob
import matplotlib.pyplot as plt
import ast
# import moss
import csv
import random
%matplotlib inline
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import scipy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning

In [3]:
df = pd.read_csv('./pilot3_nonresp_inc.csv')
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns


In [4]:
def remove_unit_variance(df, col, unit, group=None, suffix="_within"):
    """Remove variance between sampling units.

    This is useful for plotting repeated-measures data using within-unit
    error bars.

    Parameters
    ----------
    df : DataFrame
        Input data. Will have a new column added.
    col : column name
        Column in dataframe with quantitative measure to modify.
    unit : column name
        Column in dataframe defining sampling units (e.g., subjects).
    group : column name(s), optional
        Columns defining groups to remove unit variance within.
    suffix : string, optional
        Suffix appended to ``col`` name to create new column.

    Returns
    -------
    df : DataFrame
        Returns modified dataframe.

    """
    new_col = col + suffix

    def demean(x):
        return x - x.mean()

    if group is None:
        new = df.groupby(unit)[col].transform(demean)
        new += df[col].mean()
        df.loc[:, new_col] = new
    else:
        df.loc[:, new_col] = np.nan
        for level, df_level in df.groupby(group):
            new = df_level.groupby(unit)[col].transform(demean)
            new += df_level[col].mean()
            df.loc[new.index, new_col] = new

    return df

In [5]:
df = remove_unit_variance(df,'mouse.time_last','participant')
df['analysis_rt'] = df['mouse.time_last_within']

In [6]:
df['IT Distractor Similarity\nto Prioritized Item']

0                  NaN
1                     
2                  NaN
3                     
4                     
             ...      
41695              NaN
41696    Least Similar
41697                 
41698    Least Similar
41699                 
Name: IT Distractor Similarity\nto Prioritized Item, Length: 41700, dtype: object

In [7]:
df['Distractor V2 Similarity Preference'] = df['v2_prefers'] 
df['Distractor IT Similarity Preference'] = df['it_prefers']


column_params = {
    'v2': {'n_cats': 5, 'labels': ['Least Similar', '', ' ', '   ', 'Most Similar']},
    'it': {'n_cats': 5, 'labels': ['Least Similar', '', ' ', '   ', 'Most Similar']}
}

for label in ['it_sim_dis_attend', 'v2_sim_dis_attend', 'it_sim_dis_unattend', 'v2_sim_dis_unattend', 'it_sim_dis_diff', 'v2_sim_dis_diff','v2_sim_dis_diff_sq', 'it_sim_dis_diff_sq']:
    
    if 'v2' in label:
        n_cats = column_params['v2']['n_cats']
        labels = column_params['v2']['labels']
    else:
        n_cats = column_params['it']['n_cats']
        labels = column_params['it']['labels']
    
    df[label + '_cat'] = pd.qcut(df[label], q=n_cats, labels=labels, duplicates='drop')


df['V2 Distractor Similarity\nto Prioritized Item'] = df['v2_sim_dis_attend_cat']
df['IT Distractor Similarity\nto Prioritized Item'] = df['it_sim_dis_attend_cat']
df['V2 Distractor Similarity\nto Deprioritized Item'] = df['v2_sim_dis_unattend_cat']
df['IT Distractor Similarity\nto Deprioritized Item'] = df['it_sim_dis_unattend_cat']
df['Prioritized - Deprioritized IT Distractor Similarity'] = df['it_sim_dis_diff_cat'] 
df['Prioritized - Deprioritized V2 Distractor Similarity'] = df['v2_sim_dis_diff_cat'] 
df['Prioritized - Deprioritized IT Distractor Similarity Squared'] = df['v2_sim_dis_diff_sq_cat'] 
df['Prioritized - Deprioritized V2 Distractor Similarity Squared'] = df['it_sim_dis_diff_sq_cat'] 


In [8]:
df['it_sim_dis_diff'] = df['it_sim_dis_diff'] - np.mean(df['it_sim_dis_diff'])
df['v2_sim_dis_diff'] = df['v2_sim_dis_diff'] - np.mean(df['v2_sim_dis_diff'])

df['it_sim_dis_diff_sq'] = [x**2 for x in df['it_sim_dis_diff']]
df['v2_sim_dis_diff_sq'] = [x**2 for x in df['v2_sim_dis_diff']]

df['it_sim_dis_diff_sq'] = df['it_sim_dis_diff_sq'] - np.mean(df['it_sim_dis_diff_sq'])
df['v2_sim_dis_diff_sq'] = df['v2_sim_dis_diff_sq'] - np.mean(df['v2_sim_dis_diff_sq'])


df['IT_diff_binned'] = pd.qcut(df['it_sim_dis_diff'], 5, duplicates='drop')
df['V2_diff_binned'] = pd.qcut(df['v2_sim_dis_diff'], 5, duplicates='drop')
df['IT_diff_binned_sq'] = pd.qcut(df['it_sim_dis_diff_sq'], 5, duplicates='drop')
df['V2_diff_binned_sq'] = pd.qcut(df['v2_sim_dis_diff_sq'], 5, duplicates='drop')

def round_to_significant_figures(x, sig_figs=2):
    if x == 0:
        return 0
    else:
        return round(x, sig_figs - int(np.floor(np.log10(abs(x)))) - 1)


def process_interval(interval):
    # Round both bounds
    lower_rounded = round_to_significant_figures(interval.left)
    upper_rounded = round_to_significant_figures(interval.right)
    # Return a new interval with the rounded bounds
    return pd.Interval(lower_rounded, upper_rounded)

df['IT_diff_binned'] = df['IT_diff_binned'].apply(process_interval)
df['V2_diff_binned'] = df['V2_diff_binned'].apply(process_interval)
df['IT_diff_binned_sq'] = df['IT_diff_binned_sq'].apply(process_interval)
df['V2_diff_binned_sq'] = df['V2_diff_binned_sq'].apply(process_interval)

df['Prioritized - Deprioritized V2 Distractor Similarity Ranges'] = df['V2_diff_binned']
df['Prioritized - Deprioritized IT Distractor Similarity Ranges'] = df['IT_diff_binned']
df['Prioritized - Deprioritized V2 Distractor Similarity Squared Ranges'] = df['V2_diff_binned_sq']
df['Prioritized - Deprioritized IT Distractor Similarity Squared Ranges'] = df['IT_diff_binned_sq']

In [9]:
df['retrocue_reliability'] = df['Retrocue Reliability']
df['tested_item'] = df['Tested Item']
df_correct = df[df['resp_correct'] == 1]


In [10]:

# g = sns.catplot(x = 'Retrocue Reliability',
#             y = 'Accuracy',
#             errorbar=('ci', 68),
# #               join = False,
# #             linestyle='none',
#             hue = 'Tested Item',
# #             col = 'participant',#remove to see each participant
#             dodge = .1,
#             kind = 'point',
#             palette = palette,
#             data = df)
# g.set(ylim=(0.62, 0.8))  # Restrict the y-axis to be from 0 to 0.8

# sns.despine()

In [54]:
md = smf.mixedlm("resp_correct ~ retrocue_reliability*tested_item", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                              Mixed Linear Model Regression Results
=================================================================================================
Model:                         MixedLM              Dependent Variable:              resp_correct
No. Observations:              41700                Method:                          REML        
No. Groups:                    139                  Scale:                           0.1844      
Min. group size:               300                  Log-Likelihood:                  -24156.9946 
Max. group size:               300                  Converged:                       Yes         
Mean group size:               300.0                                                             
-------------------------------------------------------------------------------------------------
                                                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------
Intercept                                               0.638    0.012 51.172 0.000  0.614  0.663
retrocue_reliability[T.low]                             0.065    0.009  7.257 0.000  0.047  0.082
tested_item[T.prioritized]                              0.112    0.008 14.093 0.000  0.097  0.128
retrocue_reliability[T.low]:tested_item[T.prioritized] -0.091    0.010 -8.941 0.000 -0.111 -0.071
Group Var                                               0.014    0.004                           
=================================================================================================

"""

In [12]:
md = smf.mixedlm("resp_correct ~ v2_sim_dis_attend_cat", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                     Mixed Linear Model Regression Results
================================================================================
Model:                    MixedLM        Dependent Variable:        resp_correct
No. Observations:         41700          Method:                    REML        
No. Groups:               139            Scale:                     0.1850      
Min. group size:          300            Log-Likelihood:            -24225.1368 
Max. group size:          300            Converged:                 Yes         
Mean group size:          300.0                                                 
--------------------------------------------------------------------------------
                                      Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------------
Intercept                              0.720    0.011 64.516 0.000  0.698  0.742
v2_sim_dis_attend_cat[T.]              0.015    0.007  2.286 0.022  0.002  0.028
v2_sim_dis_attend_cat[T. ]            -0.018    0.007 -2.772 0.006 -0.032 -0.005
v2_sim_dis_attend_cat[T.   ]           0.038    0.007  5.689 0.000  0.025  0.051
v2_sim_dis_attend_cat[T.Most Similar] -0.013    0.007 -1.980 0.048 -0.026 -0.000
Group Var                              0.014    0.004                           
================================================================================

"""

In [13]:
md = smf.mixedlm("resp_correct ~ v2_sim_dis_attend_cat*tested_item", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                   Mixed Linear Model Regression Results
===========================================================================================================
Model:                             MixedLM                 Dependent Variable:                 resp_correct
No. Observations:                  41700                   Method:                             REML        
No. Groups:                        139                     Scale:                              0.1843      
Min. group size:                   300                     Log-Likelihood:                     -24168.5315 
Max. group size:                   300                     Converged:                          Yes         
Mean group size:                   300.0                                                                   
-----------------------------------------------------------------------------------------------------------
                                                                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                                         0.668    0.014 48.352 0.000  0.641  0.695
v2_sim_dis_attend_cat[T.]                                         0.025    0.013  2.003 0.045  0.001  0.050
v2_sim_dis_attend_cat[T. ]                                       -0.005    0.013 -0.385 0.700 -0.030  0.020
v2_sim_dis_attend_cat[T.   ]                                      0.051    0.014  3.792 0.000  0.025  0.078
v2_sim_dis_attend_cat[T.Most Similar]                            -0.011    0.015 -0.758 0.448 -0.040  0.018
tested_item[T.prioritized]                                        0.070    0.011  6.420 0.000  0.048  0.091
v2_sim_dis_attend_cat[T.]:tested_item[T.prioritized]             -0.008    0.015 -0.525 0.600 -0.037  0.021
v2_sim_dis_attend_cat[T. ]:tested_item[T.prioritized]            -0.016    0.015 -1.036 0.300 -0.045  0.014
v2_sim_dis_attend_cat[T.   ]:tested_item[T.prioritized]          -0.019    0.016 -1.225 0.221 -0.049  0.011
v2_sim_dis_attend_cat[T.Most Similar]:tested_item[T.prioritized] -0.009    0.017 -0.557 0.577 -0.042  0.023
Group Var                                                         0.014    0.004                           
===========================================================================================================

"""

In [14]:
md = smf.mixedlm("resp_correct ~ v2_sim_dis_attend_cat*retrocue_reliability", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                   Mixed Linear Model Regression Results
============================================================================================================
Model:                              MixedLM                 Dependent Variable:                 resp_correct
No. Observations:                   41700                   Method:                             REML        
No. Groups:                         139                     Scale:                              0.1848      
Min. group size:                    300                     Log-Likelihood:                     -24219.3175 
Max. group size:                    300                     Converged:                          Yes         
Mean group size:                    300.0                                                                   
------------------------------------------------------------------------------------------------------------
                                                                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                                          0.752    0.012 61.367 0.000  0.728  0.776
v2_sim_dis_attend_cat[T.]                                         -0.006    0.010 -0.624 0.533 -0.025  0.013
v2_sim_dis_attend_cat[T. ]                                        -0.049    0.009 -5.243 0.000 -0.067 -0.031
v2_sim_dis_attend_cat[T.   ]                                      -0.002    0.010 -0.192 0.848 -0.021  0.017
v2_sim_dis_attend_cat[T.Most Similar]                             -0.036    0.010 -3.753 0.000 -0.055 -0.017
retrocue_reliability[T.low]                                       -0.058    0.009 -6.192 0.000 -0.077 -0.040
v2_sim_dis_attend_cat[T.]:retrocue_reliability[T.low]              0.039    0.013  2.890 0.004  0.012  0.065
v2_sim_dis_attend_cat[T. ]:retrocue_reliability[T.low]             0.057    0.013  4.289 0.000  0.031  0.083
v2_sim_dis_attend_cat[T.   ]:retrocue_reliability[T.low]           0.075    0.013  5.628 0.000  0.049  0.101
v2_sim_dis_attend_cat[T.Most Similar]:retrocue_reliability[T.low]  0.042    0.013  3.124 0.002  0.016  0.068
Group Var                                                          0.014    0.004                           
============================================================================================================

"""

In [15]:
md = smf.mixedlm("resp_correct ~ v2_sim_dis_attend_cat*tested_item*retrocue_reliability", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                                 Mixed Linear Model Regression Results
=======================================================================================================================================
Model:                                       MixedLM                          Dependent Variable:                          resp_correct
No. Observations:                            41700                            Method:                                      REML        
No. Groups:                                  139                              Scale:                                       0.1834      
Min. group size:                             300                              Log-Likelihood:                              -24088.6890 
Max. group size:                             300                              Converged:                                   Yes         
Mean group size:                             300.0                                                                                     
---------------------------------------------------------------------------------------------------------------------------------------
                                                                                             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                     0.659    0.019 34.427 0.000  0.621  0.697
v2_sim_dis_attend_cat[T.]                                                                     0.028    0.022  1.276 0.202 -0.015  0.071
v2_sim_dis_attend_cat[T. ]                                                                   -0.154    0.023 -6.702 0.000 -0.199 -0.109
v2_sim_dis_attend_cat[T.   ]                                                                  0.057    0.024  2.333 0.020  0.009  0.105
v2_sim_dis_attend_cat[T.Most Similar]                                                        -0.029    0.023 -1.253 0.210 -0.074  0.016
tested_item[T.prioritized]                                                                    0.113    0.018  6.286 0.000  0.078  0.148
retrocue_reliability[T.low]                                                                   0.014    0.020  0.687 0.492 -0.025  0.053
v2_sim_dis_attend_cat[T.]:tested_item[T.prioritized]                                         -0.040    0.024 -1.619 0.105 -0.087  0.008
v2_sim_dis_attend_cat[T. ]:tested_item[T.prioritized]                                         0.120    0.025  4.771 0.000  0.071  0.169
v2_sim_dis_attend_cat[T.   ]:tested_item[T.prioritized]                                      -0.073    0.027 -2.766 0.006 -0.125 -0.021
v2_sim_dis_attend_cat[T.Most Similar]:tested_item[T.prioritized]                             -0.010    0.025 -0.405 0.686 -0.060  0.039
v2_sim_dis_attend_cat[T.]:retrocue_reliability[T.low]                                        -0.005    0.027 -0.169 0.865 -0.057  0.048
v2_sim_dis_attend_cat[T. ]:retrocue_reliability[T.low]                                        0.210    0.028  7.581 0.000  0.156  0.265
v2_sim_dis_attend_cat[T.   ]:retrocue_reliability[T.low]                                     -0.009    0.029 -0.295 0.768 -0.066  0.049
v2_sim_dis_attend_cat[T.Most Similar]:retrocue_reliability[T.low]                             0.040    0.030  1.302 0.193 -0.020  0.099
tested_item[T.prioritized]:retrocue_reliability[T.low]                                       -0.083    0.023 -3.660 0.000 -0.127 -0.039
v2_sim_dis_attend_cat[T.]:tested_item[T.prioritized]:retrocue_reliability[T.low]              0.062    0.031  1.987 0.047  0.001  0.123
v2_sim_dis_attend_cat[T. ]:tested_item[T.prioritized]:retrocue_reliability[T.low]            -0.200    0.032 -6.243 0.000 -0.262 -0.137
v2_sim_dis_attend_cat[T.   ]:tested_item[T.prioritized]:

In [16]:
md = smf.mixedlm("resp_correct ~ it_sim_dis_attend_cat", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                     Mixed Linear Model Regression Results
================================================================================
Model:                    MixedLM        Dependent Variable:        resp_correct
No. Observations:         41700          Method:                    REML        
No. Groups:               139            Scale:                     0.1851      
Min. group size:          300            Log-Likelihood:            -24241.1724 
Max. group size:          300            Converged:                 Yes         
Mean group size:          300.0                                                 
--------------------------------------------------------------------------------
                                      Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------------
Intercept                              0.717    0.011 64.223 0.000  0.695  0.739
it_sim_dis_attend_cat[T.]             -0.003    0.007 -0.468 0.640 -0.016  0.010
it_sim_dis_attend_cat[T. ]             0.033    0.007  4.895 0.000  0.020  0.046
it_sim_dis_attend_cat[T.   ]           0.021    0.007  3.113 0.002  0.008  0.034
it_sim_dis_attend_cat[T.Most Similar] -0.013    0.007 -1.890 0.059 -0.026  0.000
Group Var                              0.014    0.004                           
================================================================================

"""

In [17]:
md = smf.mixedlm("resp_correct ~ it_sim_dis_attend_cat*tested_item", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                   Mixed Linear Model Regression Results
===========================================================================================================
Model:                             MixedLM                 Dependent Variable:                 resp_correct
No. Observations:                  41700                   Method:                             REML        
No. Groups:                        139                     Scale:                              0.1845      
Min. group size:                   300                     Log-Likelihood:                     -24186.0474 
Max. group size:                   300                     Converged:                          Yes         
Mean group size:                   300.0                                                                   
-----------------------------------------------------------------------------------------------------------
                                                                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                                         0.678    0.014 48.252 0.000  0.650  0.705
it_sim_dis_attend_cat[T.]                                         0.009    0.013  0.682 0.495 -0.017  0.034
it_sim_dis_attend_cat[T. ]                                        0.036    0.015  2.435 0.015  0.007  0.065
it_sim_dis_attend_cat[T.   ]                                      0.007    0.013  0.559 0.576 -0.018  0.033
it_sim_dis_attend_cat[T.Most Similar]                            -0.029    0.014 -2.127 0.033 -0.056 -0.002
tested_item[T.prioritized]                                        0.051    0.011  4.615 0.000  0.030  0.073
it_sim_dis_attend_cat[T.]:tested_item[T.prioritized]             -0.012    0.015 -0.807 0.420 -0.042  0.017
it_sim_dis_attend_cat[T. ]:tested_item[T.prioritized]            -0.007    0.016 -0.423 0.672 -0.039  0.025
it_sim_dis_attend_cat[T.   ]:tested_item[T.prioritized]           0.024    0.015  1.597 0.110 -0.005  0.054
it_sim_dis_attend_cat[T.Most Similar]:tested_item[T.prioritized]  0.022    0.016  1.385 0.166 -0.009  0.053
Group Var                                                         0.014    0.004                           
===========================================================================================================

"""

In [18]:
md = smf.mixedlm("resp_correct ~ it_sim_dis_attend_cat*retrocue_reliability", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                   Mixed Linear Model Regression Results
============================================================================================================
Model:                              MixedLM                 Dependent Variable:                 resp_correct
No. Observations:                   41700                   Method:                             REML        
No. Groups:                         139                     Scale:                              0.1850      
Min. group size:                    300                     Log-Likelihood:                     -24242.7492 
Max. group size:                    300                     Converged:                          Yes         
Mean group size:                    300.0                                                                   
------------------------------------------------------------------------------------------------------------
                                                                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                                          0.717    0.012 58.855 0.000  0.693  0.741
it_sim_dis_attend_cat[T.]                                          0.016    0.009  1.741 0.082 -0.002  0.035
it_sim_dis_attend_cat[T. ]                                         0.050    0.010  5.054 0.000  0.030  0.069
it_sim_dis_attend_cat[T.   ]                                       0.030    0.009  3.229 0.001  0.012  0.048
it_sim_dis_attend_cat[T.Most Similar]                             -0.019    0.010 -1.960 0.050 -0.037 -0.000
retrocue_reliability[T.low]                                        0.000    0.009  0.042 0.967 -0.018  0.019
it_sim_dis_attend_cat[T.]:retrocue_reliability[T.low]             -0.040    0.013 -3.028 0.002 -0.066 -0.014
it_sim_dis_attend_cat[T. ]:retrocue_reliability[T.low]            -0.030    0.013 -2.268 0.023 -0.057 -0.004
it_sim_dis_attend_cat[T.   ]:retrocue_reliability[T.low]          -0.021    0.013 -1.552 0.121 -0.047  0.005
it_sim_dis_attend_cat[T.Most Similar]:retrocue_reliability[T.low]  0.012    0.013  0.906 0.365 -0.014  0.038
Group Var                                                          0.014    0.004                           
============================================================================================================

"""

In [19]:
md = smf.mixedlm("resp_correct ~ v2_sim_dis_attend_cat*tested_item*retrocue_reliability", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                                 Mixed Linear Model Regression Results
=======================================================================================================================================
Model:                                       MixedLM                          Dependent Variable:                          resp_correct
No. Observations:                            41700                            Method:                                      REML        
No. Groups:                                  139                              Scale:                                       0.1834      
Min. group size:                             300                              Log-Likelihood:                              -24088.6890 
Max. group size:                             300                              Converged:                                   Yes         
Mean group size:                             300.0                                                                                     
---------------------------------------------------------------------------------------------------------------------------------------
                                                                                             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                     0.659    0.019 34.427 0.000  0.621  0.697
v2_sim_dis_attend_cat[T.]                                                                     0.028    0.022  1.276 0.202 -0.015  0.071
v2_sim_dis_attend_cat[T. ]                                                                   -0.154    0.023 -6.702 0.000 -0.199 -0.109
v2_sim_dis_attend_cat[T.   ]                                                                  0.057    0.024  2.333 0.020  0.009  0.105
v2_sim_dis_attend_cat[T.Most Similar]                                                        -0.029    0.023 -1.253 0.210 -0.074  0.016
tested_item[T.prioritized]                                                                    0.113    0.018  6.286 0.000  0.078  0.148
retrocue_reliability[T.low]                                                                   0.014    0.020  0.687 0.492 -0.025  0.053
v2_sim_dis_attend_cat[T.]:tested_item[T.prioritized]                                         -0.040    0.024 -1.619 0.105 -0.087  0.008
v2_sim_dis_attend_cat[T. ]:tested_item[T.prioritized]                                         0.120    0.025  4.771 0.000  0.071  0.169
v2_sim_dis_attend_cat[T.   ]:tested_item[T.prioritized]                                      -0.073    0.027 -2.766 0.006 -0.125 -0.021
v2_sim_dis_attend_cat[T.Most Similar]:tested_item[T.prioritized]                             -0.010    0.025 -0.405 0.686 -0.060  0.039
v2_sim_dis_attend_cat[T.]:retrocue_reliability[T.low]                                        -0.005    0.027 -0.169 0.865 -0.057  0.048
v2_sim_dis_attend_cat[T. ]:retrocue_reliability[T.low]                                        0.210    0.028  7.581 0.000  0.156  0.265
v2_sim_dis_attend_cat[T.   ]:retrocue_reliability[T.low]                                     -0.009    0.029 -0.295 0.768 -0.066  0.049
v2_sim_dis_attend_cat[T.Most Similar]:retrocue_reliability[T.low]                             0.040    0.030  1.302 0.193 -0.020  0.099
tested_item[T.prioritized]:retrocue_reliability[T.low]                                       -0.083    0.023 -3.660 0.000 -0.127 -0.039
v2_sim_dis_attend_cat[T.]:tested_item[T.prioritized]:retrocue_reliability[T.low]              0.062    0.031  1.987 0.047  0.001  0.123
v2_sim_dis_attend_cat[T. ]:tested_item[T.prioritized]:retrocue_reliability[T.low]            -0.200    0.032 -6.243 0.000 -0.262 -0.137
v2_sim_dis_attend_cat[T.   ]:tested_item[T.prioritized]:

In [20]:

md = smf.mixedlm("resp_correct ~ V2_diff_binned", df, 
                 groups=df["participant"]).fit()
md.summary()


<class 'statsmodels.iolib.summary2.Summary'>
"""
                               Mixed Linear Model Regression Results
===================================================================================================
Model:                           MixedLM              Dependent Variable:              resp_correct
No. Observations:                41700                Method:                          REML        
No. Groups:                      139                  Scale:                           0.1850      
Min. group size:                 300                  Log-Likelihood:                  -24230.5692 
Max. group size:                 300                  Converged:                       Yes         
Mean group size:                 300.0                                                             
---------------------------------------------------------------------------------------------------
                                                          Coef. Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------------------------------
Intercept                                                 0.705    0.011 63.151 0.000  0.683  0.727
V2_diff_binned[T.Interval(-0.1, -0.039, closed='right')]  0.015    0.007  2.268 0.023  0.002  0.028
V2_diff_binned[T.Interval(-0.039, 0.036, closed='right')] 0.054    0.007  8.100 0.000  0.041  0.067
V2_diff_binned[T.Interval(0.036, 0.096, closed='right')]  0.024    0.007  3.672 0.000  0.011  0.038
V2_diff_binned[T.Interval(0.096, 0.17, closed='right')]   0.004    0.007  0.612 0.541 -0.009  0.017
Group Var                                                 0.014    0.004                           
===================================================================================================

"""

In [21]:

md = smf.mixedlm("resp_correct ~ V2_diff_binned* tested_item", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                             Mixed Linear Model Regression Results
===============================================================================================================================
Model:                                   MixedLM                        Dependent Variable:                        resp_correct
No. Observations:                        41700                          Method:                                    REML        
No. Groups:                              139                            Scale:                                     0.1842      
Min. group size:                         300                            Log-Likelihood:                            -24148.2087 
Max. group size:                         300                            Converged:                                 Yes         
Mean group size:                         300.0                                                                                 
-------------------------------------------------------------------------------------------------------------------------------
                                                                                     Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                             0.656    0.014 46.699 0.000  0.628  0.683
V2_diff_binned[T.Interval(-0.1, -0.039, closed='right')]                              0.014    0.013  1.081 0.279 -0.012  0.040
V2_diff_binned[T.Interval(-0.039, 0.036, closed='right')]                             0.045    0.013  3.549 0.000  0.020  0.070
V2_diff_binned[T.Interval(0.036, 0.096, closed='right')]                              0.071    0.014  5.098 0.000  0.044  0.099
V2_diff_binned[T.Interval(0.096, 0.17, closed='right')]                              -0.013    0.015 -0.873 0.383 -0.042  0.016
tested_item[T.prioritized]                                                            0.065    0.011  5.806 0.000  0.043  0.086
V2_diff_binned[T.Interval(-0.1, -0.039, closed='right')]:tested_item[T.prioritized]   0.006    0.015  0.378 0.706 -0.024  0.036
V2_diff_binned[T.Interval(-0.039, 0.036, closed='right')]:tested_item[T.prioritized]  0.023    0.015  1.547 0.122 -0.006  0.052
V2_diff_binned[T.Interval(0.036, 0.096, closed='right')]:tested_item[T.prioritized]  -0.061    0.016 -3.855 0.000 -0.093 -0.030
V2_diff_binned[T.Interval(0.096, 0.17, closed='right')]:tested_item[T.prioritized]    0.017    0.016  1.016 0.310 -0.016  0.049
Group Var                                                                             0.014    0.004                           
===============================================================================================================================

"""

In [22]:

md = smf.mixedlm("resp_correct ~ V2_diff_binned*retrocue_reliability", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                             Mixed Linear Model Regression Results
================================================================================================================================
Model:                                    MixedLM                        Dependent Variable:                        resp_correct
No. Observations:                         41700                          Method:                                    REML        
No. Groups:                               139                            Scale:                                     0.1850      
Min. group size:                          300                            Log-Likelihood:                            -24238.5600 
Max. group size:                          300                            Converged:                                 Yes         
Mean group size:                          300.0                                                                                 
--------------------------------------------------------------------------------------------------------------------------------
                                                                                      Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                              0.706    0.012 58.298 0.000  0.683  0.730
V2_diff_binned[T.Interval(-0.1, -0.039, closed='right')]                               0.029    0.010  3.067 0.002  0.011  0.048
V2_diff_binned[T.Interval(-0.039, 0.036, closed='right')]                              0.059    0.009  6.453 0.000  0.041  0.077
V2_diff_binned[T.Interval(0.036, 0.096, closed='right')]                               0.019    0.010  2.021 0.043  0.001  0.038
V2_diff_binned[T.Interval(0.096, 0.17, closed='right')]                                0.016    0.009  1.734 0.083 -0.002  0.035
retrocue_reliability[T.low]                                                           -0.003    0.009 -0.280 0.779 -0.021  0.016
V2_diff_binned[T.Interval(-0.1, -0.039, closed='right')]:retrocue_reliability[T.low]  -0.027    0.013 -1.996 0.046 -0.053 -0.000
V2_diff_binned[T.Interval(-0.039, 0.036, closed='right')]:retrocue_reliability[T.low] -0.012    0.013 -0.925 0.355 -0.039  0.014
V2_diff_binned[T.Interval(0.036, 0.096, closed='right')]:retrocue_reliability[T.low]   0.010    0.013  0.728 0.467 -0.016  0.036
V2_diff_binned[T.Interval(0.096, 0.17, closed='right')]:retrocue_reliability[T.low]   -0.025    0.013 -1.889 0.059 -0.051  0.001
Group Var                                                                              0.014    0.004                           
================================================================================================================================

"""

In [23]:
md = smf.mixedlm("resp_correct ~ V2_diff_binned*tested_item*retrocue_reliability", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                                           Mixed Linear Model Regression Results
===========================================================================================================================================================
Model:                                             MixedLM                                 Dependent Variable:                                 resp_correct
No. Observations:                                  41700                                   Method:                                             REML        
No. Groups:                                        139                                     Scale:                                              0.1835      
Min. group size:                                   300                                     Log-Likelihood:                                     -24098.5213 
Max. group size:                                   300                                     Converged:                                          Yes         
Mean group size:                                   300.0                                                                                                   
-----------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                                                                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                         0.658    0.021 31.652 0.000  0.618  0.699
V2_diff_binned[T.Interval(-0.1, -0.039, closed='right')]                                                         -0.084    0.024 -3.454 0.001 -0.132 -0.036
V2_diff_binned[T.Interval(-0.039, 0.036, closed='right')]                                                        -0.024    0.022 -1.091 0.275 -0.068  0.019
V2_diff_binned[T.Interval(0.036, 0.096, closed='right')]                                                          0.083    0.028  2.982 0.003  0.028  0.137
V2_diff_binned[T.Interval(0.096, 0.17, closed='right')]                                                          -0.027    0.024 -1.092 0.275 -0.074  0.021
tested_item[T.prioritized]                                                                                        0.056    0.020  2.850 0.004  0.017  0.094
retrocue_reliability[T.low]                                                                                      -0.004    0.021 -0.167 0.867 -0.046  0.039
V2_diff_binned[T.Interval(-0.1, -0.039, closed='right')]:tested_item[T.prioritized]                               0.141    0.026  5.333 0.000  0.089  0.193
V2_diff_binned[T.Interval(-0.039, 0.036, closed='right')]:tested_item[T.prioritized]                              0.118    0.024  4.841 0.000  0.070  0.166
V2_diff_binned[T.Interval(0.036, 0.096, closed='right')]:tested_item[T.prioritized]                              -0.073    0.030 -2.456 0.014 -0.131 -0.015
V2_diff_binned[T.Interval(0.096, 0.17, closed='right')]:tested_item[T.prioritized]                                0.053    0.026  2.006 0.045  0.001  0.105
V2_diff_binned[T.Interval(-0.1, -0.039, closed='right')]:retrocue_reliability[T.low]                              0.139    0.029  4.819 0.000  0.083  0.196
V2_diff_binned[T.Interval(-0.039, 0.036, closed='right')]:retrocue_reliability[T.low]                             0.115    0.027  4.229 0.000  0.062  0.168
V2_diff_binned[T.Interval(0.036, 0.096, closed='right')]:retrocue_reliability[T.low]                             -0.014    0.032 -0.447 0.655 -0.077  0.049
V2_diff_binned[T.Interval(0.096, 0.17, closed='right')]:retrocue_reliability[T.low]                           

In [24]:

md = smf.mixedlm("resp_correct ~ IT_diff_binned", df, 
                 groups=df["participant"]).fit()
md.summary()


<class 'statsmodels.iolib.summary2.Summary'>
"""
                               Mixed Linear Model Regression Results
===================================================================================================
Model:                           MixedLM              Dependent Variable:              resp_correct
No. Observations:                41700                Method:                          REML        
No. Groups:                      139                  Scale:                           0.1847      
Min. group size:                 300                  Log-Likelihood:                  -24195.7347 
Max. group size:                 300                  Converged:                       Yes         
Mean group size:                 300.0                                                             
---------------------------------------------------------------------------------------------------
                                                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------------------------------
Intercept                                                 0.702    0.011 62.912 0.000  0.681  0.724
IT_diff_binned[T.Interval(-0.82, -0.34, closed='right')]  0.036    0.007  5.477 0.000  0.023  0.049
IT_diff_binned[T.Interval(-0.34, 0.28, closed='right')]   0.044    0.007  6.540 0.000  0.030  0.057
IT_diff_binned[T.Interval(0.28, 0.76, closed='right')]    0.049    0.007  7.296 0.000  0.036  0.062
IT_diff_binned[T.Interval(0.76, 1.3, closed='right')]    -0.017    0.007 -2.576 0.010 -0.030 -0.004
Group Var                                                 0.014    0.004                           
===================================================================================================

"""

In [25]:

md = smf.mixedlm("resp_correct ~ IT_diff_binned*tested_item", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                            Mixed Linear Model Regression Results
==============================================================================================================================
Model:                                    MixedLM                       Dependent Variable:                       resp_correct
No. Observations:                         41700                         Method:                                   REML        
No. Groups:                               139                           Scale:                                    0.1841      
Min. group size:                          300                           Log-Likelihood:                           -24137.8769 
Max. group size:                          300                           Converged:                                Yes         
Mean group size:                          300.0                                                                               
------------------------------------------------------------------------------------------------------------------------------
                                                                                    Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                            0.661    0.014 47.070 0.000  0.633  0.688
IT_diff_binned[T.Interval(-0.82, -0.34, closed='right')]                             0.043    0.013  3.359 0.001  0.018  0.068
IT_diff_binned[T.Interval(-0.34, 0.28, closed='right')]                              0.061    0.017  3.600 0.000  0.028  0.094
IT_diff_binned[T.Interval(0.28, 0.76, closed='right')]                               0.017    0.013  1.293 0.196 -0.009  0.043
IT_diff_binned[T.Interval(0.76, 1.3, closed='right')]                                0.001    0.013  0.110 0.913 -0.024  0.027
tested_item[T.prioritized]                                                           0.054    0.011  4.880 0.000  0.032  0.076
IT_diff_binned[T.Interval(-0.82, -0.34, closed='right')]:tested_item[T.prioritized] -0.003    0.015 -0.204 0.839 -0.032  0.026
IT_diff_binned[T.Interval(-0.34, 0.28, closed='right')]:tested_item[T.prioritized]  -0.027    0.018 -1.444 0.149 -0.063  0.009
IT_diff_binned[T.Interval(0.28, 0.76, closed='right')]:tested_item[T.prioritized]    0.045    0.015  2.941 0.003  0.015  0.075
IT_diff_binned[T.Interval(0.76, 1.3, closed='right')]:tested_item[T.prioritized]    -0.021    0.015 -1.368 0.171 -0.050  0.009
Group Var                                                                            0.014    0.004                           
==============================================================================================================================

"""

In [26]:

md = smf.mixedlm("resp_correct ~ IT_diff_binned*retrocue_reliability", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                             Mixed Linear Model Regression Results
===============================================================================================================================
Model:                                   MixedLM                        Dependent Variable:                        resp_correct
No. Observations:                        41700                          Method:                                    REML        
No. Groups:                              139                            Scale:                                     0.1842      
Min. group size:                         300                            Log-Likelihood:                            -24153.7751 
Max. group size:                         300                            Converged:                                 Yes         
Mean group size:                         300.0                                                                                 
-------------------------------------------------------------------------------------------------------------------------------
                                                                                     Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                             0.692    0.012 56.832 0.000  0.668  0.716
IT_diff_binned[T.Interval(-0.82, -0.34, closed='right')]                              0.086    0.009  9.189 0.000  0.067  0.104
IT_diff_binned[T.Interval(-0.34, 0.28, closed='right')]                               0.067    0.010  6.906 0.000  0.048  0.086
IT_diff_binned[T.Interval(0.28, 0.76, closed='right')]                                0.038    0.009  4.063 0.000  0.020  0.057
IT_diff_binned[T.Interval(0.76, 1.3, closed='right')]                                 0.009    0.009  0.929 0.353 -0.010  0.027
retrocue_reliability[T.low]                                                           0.020    0.009  2.163 0.031  0.002  0.039
IT_diff_binned[T.Interval(-0.82, -0.34, closed='right')]:retrocue_reliability[T.low] -0.103    0.013 -7.775 0.000 -0.130 -0.077
IT_diff_binned[T.Interval(-0.34, 0.28, closed='right')]:retrocue_reliability[T.low]  -0.044    0.013 -3.327 0.001 -0.070 -0.018
IT_diff_binned[T.Interval(0.28, 0.76, closed='right')]:retrocue_reliability[T.low]    0.023    0.013  1.715 0.086 -0.003  0.049
IT_diff_binned[T.Interval(0.76, 1.3, closed='right')]:retrocue_reliability[T.low]    -0.052    0.013 -3.921 0.000 -0.078 -0.026
Group Var                                                                             0.014    0.004                           
===============================================================================================================================

"""

In [27]:
md = smf.mixedlm("resp_correct ~ IT_diff_binned*tested_item*retrocue_reliability", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                                          Mixed Linear Model Regression Results
==========================================================================================================================================================
Model:                                            MixedLM                                 Dependent Variable:                                 resp_correct
No. Observations:                                 41700                                   Method:                                             REML        
No. Groups:                                       139                                     Scale:                                              0.1831      
Min. group size:                                  300                                     Log-Likelihood:                                     -24054.5990 
Max. group size:                                  300                                     Converged:                                          Yes         
Mean group size:                                  300.0                                                                                                   
----------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                                                                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                        0.578    0.019 30.235 0.000  0.541  0.616
IT_diff_binned[T.Interval(-0.82, -0.34, closed='right')]                                                         0.176    0.023  7.647 0.000  0.131  0.221
IT_diff_binned[T.Interval(-0.34, 0.28, closed='right')]                                                          0.123    0.030  4.051 0.000  0.064  0.183
IT_diff_binned[T.Interval(0.28, 0.76, closed='right')]                                                          -0.010    0.021 -0.474 0.636 -0.052  0.032
IT_diff_binned[T.Interval(0.76, 1.3, closed='right')]                                                            0.074    0.022  3.361 0.001  0.031  0.117
tested_item[T.prioritized]                                                                                       0.137    0.018  7.685 0.000  0.102  0.172
retrocue_reliability[T.low]                                                                                      0.128    0.020  6.334 0.000  0.089  0.168
IT_diff_binned[T.Interval(-0.82, -0.34, closed='right')]:tested_item[T.prioritized]                             -0.109    0.025 -4.342 0.000 -0.158 -0.060
IT_diff_binned[T.Interval(-0.34, 0.28, closed='right')]:tested_item[T.prioritized]                              -0.075    0.032 -2.331 0.020 -0.138 -0.012
IT_diff_binned[T.Interval(0.28, 0.76, closed='right')]:tested_item[T.prioritized]                                0.072    0.024  3.031 0.002  0.025  0.118
IT_diff_binned[T.Interval(0.76, 1.3, closed='right')]:tested_item[T.prioritized]                                -0.077    0.024 -3.175 0.001 -0.125 -0.030
IT_diff_binned[T.Interval(-0.82, -0.34, closed='right')]:retrocue_reliability[T.low]                            -0.196    0.028 -7.081 0.000 -0.250 -0.142
IT_diff_binned[T.Interval(-0.34, 0.28, closed='right')]:retrocue_reliability[T.low]                             -0.100    0.036 -2.745 0.006 -0.172 -0.029
IT_diff_binned[T.Interval(0.28, 0.76, closed='right')]:retrocue_reliability[T.low]                               0.067    0.027  2.449 0.014  0.013  0.120
IT_diff_binned[T.Interval(0.76, 1.3, closed='right')]:retrocue_reliability[T.low]                               -0.114    0.027 -4.204 

In [28]:
md = smf.mixedlm("resp_correct ~ V2_diff_binned_sq", df, 
                 groups=df["participant"]).fit()
md.summary()




<class 'statsmodels.iolib.summary2.Summary'>
"""
                                  Mixed Linear Model Regression Results
==========================================================================================================
Model:                            MixedLM                 Dependent Variable:                 resp_correct
No. Observations:                 41700                   Method:                             REML        
No. Groups:                       139                     Scale:                              0.1851      
Min. group size:                  300                     Log-Likelihood:                     -24236.6926 
Max. group size:                  300                     Converged:                          Yes         
Mean group size:                  300.0                                                                   
----------------------------------------------------------------------------------------------------------
                                                                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                                        0.756    0.011 67.724 0.000  0.734  0.778
V2_diff_binned_sq[T.Interval(-0.008, -0.0046, closed='right')]  -0.027    0.007 -4.049 0.000 -0.040 -0.014
V2_diff_binned_sq[T.Interval(-0.0046, 0.00036, closed='right')] -0.036    0.007 -5.363 0.000 -0.049 -0.023
V2_diff_binned_sq[T.Interval(0.00036, 0.0081, closed='right')]  -0.044    0.007 -6.533 0.000 -0.057 -0.030
V2_diff_binned_sq[T.Interval(0.0081, 0.019, closed='right')]    -0.052    0.007 -7.739 0.000 -0.065 -0.039
Group Var                                                        0.014    0.004                           
==========================================================================================================

"""

In [29]:

md = smf.mixedlm("resp_correct ~ V2_diff_binned_sq*tested_item", df, 
                 groups=df["participant"]).fit()
md.summary()



<class 'statsmodels.iolib.summary2.Summary'>
"""
                                                Mixed Linear Model Regression Results
=====================================================================================================================================
Model:                                     MixedLM                          Dependent Variable:                          resp_correct
No. Observations:                          41700                            Method:                                      REML        
No. Groups:                                139                              Scale:                                       0.1843      
Min. group size:                           300                              Log-Likelihood:                              -24164.6161 
Max. group size:                           300                              Converged:                                   Yes         
Mean group size:                           300.0                                                                                     
-------------------------------------------------------------------------------------------------------------------------------------
                                                                                           Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                   0.701    0.013 53.423 0.000  0.676  0.727
V2_diff_binned_sq[T.Interval(-0.008, -0.0046, closed='right')]                             -0.000    0.013 -0.004 0.997 -0.025  0.025
V2_diff_binned_sq[T.Interval(-0.0046, 0.00036, closed='right')]                            -0.014    0.013 -1.110 0.267 -0.039  0.011
V2_diff_binned_sq[T.Interval(0.00036, 0.0081, closed='right')]                             -0.032    0.012 -2.605 0.009 -0.056 -0.008
V2_diff_binned_sq[T.Interval(0.0081, 0.019, closed='right')]                               -0.076    0.014 -5.347 0.000 -0.104 -0.048
tested_item[T.prioritized]                                                                  0.080    0.010  7.962 0.000  0.061  0.100
V2_diff_binned_sq[T.Interval(-0.008, -0.0046, closed='right')]:tested_item[T.prioritized]  -0.043    0.015 -2.903 0.004 -0.072 -0.014
V2_diff_binned_sq[T.Interval(-0.0046, 0.00036, closed='right')]:tested_item[T.prioritized] -0.036    0.015 -2.439 0.015 -0.065 -0.007
V2_diff_binned_sq[T.Interval(0.00036, 0.0081, closed='right')]:tested_item[T.prioritized]  -0.021    0.015 -1.428 0.153 -0.050  0.008
V2_diff_binned_sq[T.Interval(0.0081, 0.019, closed='right')]:tested_item[T.prioritized]     0.015    0.016  0.924 0.356 -0.017  0.047
Group Var                                                                                   0.014    0.004                           
=====================================================================================================================================

"""

In [30]:

md = smf.mixedlm("resp_correct ~ V2_diff_binned_sq*retrocue_reliability", df, 
                 groups=df["participant"]).fit()
md.summary()


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                                Mixed Linear Model Regression Results
======================================================================================================================================
Model:                                      MixedLM                          Dependent Variable:                          resp_correct
No. Observations:                           41700                            Method:                                      REML        
No. Groups:                                 139                              Scale:                                       0.1849      
Min. group size:                            300                              Log-Likelihood:                              -24234.8113 
Max. group size:                            300                              Converged:                                   Yes         
Mean group size:                            300.0                                                                                     
--------------------------------------------------------------------------------------------------------------------------------------
                                                                                            Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                    0.758    0.012 63.466 0.000  0.735  0.782
V2_diff_binned_sq[T.Interval(-0.008, -0.0046, closed='right')]                              -0.001    0.010 -0.097 0.922 -0.020  0.018
V2_diff_binned_sq[T.Interval(-0.0046, 0.00036, closed='right')]                             -0.047    0.009 -5.328 0.000 -0.065 -0.030
V2_diff_binned_sq[T.Interval(0.00036, 0.0081, closed='right')]                              -0.040    0.009 -4.246 0.000 -0.058 -0.021
V2_diff_binned_sq[T.Interval(0.0081, 0.019, closed='right')]                                -0.039    0.009 -4.303 0.000 -0.057 -0.021
retrocue_reliability[T.low]                                                                 -0.005    0.009 -0.520 0.603 -0.023  0.014
V2_diff_binned_sq[T.Interval(-0.008, -0.0046, closed='right')]:retrocue_reliability[T.low]  -0.042    0.013 -3.124 0.002 -0.069 -0.016
V2_diff_binned_sq[T.Interval(-0.0046, 0.00036, closed='right')]:retrocue_reliability[T.low]  0.027    0.013  2.009 0.045  0.001  0.053
V2_diff_binned_sq[T.Interval(0.00036, 0.0081, closed='right')]:retrocue_reliability[T.low]  -0.006    0.013 -0.466 0.641 -0.032  0.020
V2_diff_binned_sq[T.Interval(0.0081, 0.019, closed='right')]:retrocue_reliability[T.low]    -0.025    0.013 -1.881 0.060 -0.051  0.001
Group Var                                                                                    0.014    0.004                           
======================================================================================================================================

"""

In [31]:
md = smf.mixedlm("resp_correct ~ V2_diff_binned_sq*tested_item*retrocue_reliability", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                                              Mixed Linear Model Regression Results
=================================================================================================================================================================
Model:                                               MixedLM                                   Dependent Variable:                                   resp_correct
No. Observations:                                    41700                                     Method:                                               REML        
No. Groups:                                          139                                       Scale:                                                0.1837      
Min. group size:                                     300                                       Log-Likelihood:                                       -24129.0307 
Max. group size:                                     300                                       Converged:                                            Yes         
Mean group size:                                     300.0                                                                                                       
-----------------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                                                                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                               0.616    0.017 36.065 0.000  0.582  0.649
V2_diff_binned_sq[T.Interval(-0.008, -0.0046, closed='right')]                                                          0.068    0.025  2.704 0.007  0.019  0.117
V2_diff_binned_sq[T.Interval(-0.0046, 0.00036, closed='right')]                                                         0.019    0.020  0.923 0.356 -0.021  0.058
V2_diff_binned_sq[T.Interval(0.00036, 0.0081, closed='right')]                                                          0.061    0.021  2.848 0.004  0.019  0.102
V2_diff_binned_sq[T.Interval(0.0081, 0.019, closed='right')]                                                           -0.013    0.023 -0.575 0.565 -0.058  0.032
tested_item[T.prioritized]                                                                                              0.181    0.015 11.709 0.000  0.151  0.212
retrocue_reliability[T.low]                                                                                             0.136    0.017  7.841 0.000  0.102  0.169
V2_diff_binned_sq[T.Interval(-0.008, -0.0046, closed='right')]:tested_item[T.prioritized]                              -0.097    0.027 -3.545 0.000 -0.150 -0.043
V2_diff_binned_sq[T.Interval(-0.0046, 0.00036, closed='right')]:tested_item[T.prioritized]                             -0.088    0.023 -3.915 0.000 -0.132 -0.044
V2_diff_binned_sq[T.Interval(0.00036, 0.0081, closed='right')]:tested_item[T.prioritized]                              -0.130    0.024 -5.472 0.000 -0.176 -0.083
V2_diff_binned_sq[T.Interval(0.0081, 0.019, closed='right')]:tested_item[T.prioritized]                                -0.047    0.025 -1.902 0.057 -0.096  0.001
V2_diff_binned_sq[T.Interval(-0.008, -0.0046, closed='right')]:retrocue_reliability[T.low]                             -0.113    0.029 -3.890 0.000 -0.171 -0.056
V2_diff_binned_sq[T.Interval(-0.0046, 0.00036, closed='right')]:retrocue_reliability[T.low]                            -0.047    0.026 -1.831 0.067 -0.098  0.003
V2_diff_binned_sq[T.Interval(0.00036, 0.0081, closed='right')]:retrocue_reliability[T.low]                             -0.146

In [32]:
md = smf.mixedlm("resp_correct ~ IT_diff_binned_sq", df, 
                 groups=df["participant"]).fit()
md.summary()




<class 'statsmodels.iolib.summary2.Summary'>
"""
                                 Mixed Linear Model Regression Results
=======================================================================================================
Model:                           MixedLM                Dependent Variable:                resp_correct
No. Observations:                41700                  Method:                            REML        
No. Groups:                      139                    Scale:                             0.1848      
Min. group size:                 300                    Log-Likelihood:                    -24204.1096 
Max. group size:                 300                    Converged:                         Yes         
Mean group size:                 300.0                                                                 
-------------------------------------------------------------------------------------------------------
                                                            Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------------
Intercept                                                    0.753    0.011  67.485 0.000  0.732  0.775
IT_diff_binned_sq[T.Interval(-0.47, -0.26, closed='right')] -0.022    0.007  -3.314 0.001 -0.035 -0.009
IT_diff_binned_sq[T.Interval(-0.26, 0.05, closed='right')]  -0.012    0.007  -1.873 0.061 -0.026  0.001
IT_diff_binned_sq[T.Interval(0.05, 0.48, closed='right')]   -0.071    0.007 -10.681 0.000 -0.084 -0.058
IT_diff_binned_sq[T.Interval(0.48, 1.0, closed='right')]    -0.038    0.007  -5.764 0.000 -0.051 -0.025
Group Var                                                    0.014    0.004                            
=======================================================================================================

"""

In [33]:

md = smf.mixedlm("resp_correct ~ IT_diff_binned_sq*tested_item", df, 
                 groups=df["participant"]).fit()
md.summary()



<class 'statsmodels.iolib.summary2.Summary'>
"""
                                              Mixed Linear Model Regression Results
=================================================================================================================================
Model:                                     MixedLM                        Dependent Variable:                        resp_correct
No. Observations:                          41700                          Method:                                    REML        
No. Groups:                                139                            Scale:                                     0.1842      
Min. group size:                           300                            Log-Likelihood:                            -24150.8472 
Max. group size:                           300                            Converged:                                 Yes         
Mean group size:                           300.0                                                                                 
---------------------------------------------------------------------------------------------------------------------------------
                                                                                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                               0.721    0.017 42.235 0.000  0.688  0.755
IT_diff_binned_sq[T.Interval(-0.47, -0.26, closed='right')]                            -0.026    0.016 -1.621 0.105 -0.057  0.005
IT_diff_binned_sq[T.Interval(-0.26, 0.05, closed='right')]                             -0.058    0.017 -3.374 0.001 -0.092 -0.024
IT_diff_binned_sq[T.Interval(0.05, 0.48, closed='right')]                              -0.059    0.016 -3.559 0.000 -0.091 -0.026
IT_diff_binned_sq[T.Interval(0.48, 1.0, closed='right')]                               -0.044    0.016 -2.740 0.006 -0.076 -0.013
tested_item[T.prioritized]                                                              0.036    0.015  2.474 0.013  0.008  0.065
IT_diff_binned_sq[T.Interval(-0.47, -0.26, closed='right')]:tested_item[T.prioritized]  0.020    0.018  1.130 0.258 -0.015  0.054
IT_diff_binned_sq[T.Interval(-0.26, 0.05, closed='right')]:tested_item[T.prioritized]   0.061    0.019  3.260 0.001  0.024  0.098
IT_diff_binned_sq[T.Interval(0.05, 0.48, closed='right')]:tested_item[T.prioritized]   -0.009    0.018 -0.525 0.600 -0.045  0.026
IT_diff_binned_sq[T.Interval(0.48, 1.0, closed='right')]:tested_item[T.prioritized]     0.018    0.018  1.015 0.310 -0.017  0.053
Group Var                                                                               0.014    0.004                           
=================================================================================================================================

"""

In [34]:

md = smf.mixedlm("resp_correct ~ IT_diff_binned_sq*retrocue_reliability", df, 
                 groups=df["participant"]).fit()
md.summary()


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                              Mixed Linear Model Regression Results
==================================================================================================================================
Model:                                    MixedLM                         Dependent Variable:                         resp_correct
No. Observations:                         41700                           Method:                                     REML        
No. Groups:                               139                             Scale:                                      0.1847      
Min. group size:                          300                             Log-Likelihood:                             -24212.2091 
Max. group size:                          300                             Converged:                                  Yes         
Mean group size:                          300.0                                                                                   
----------------------------------------------------------------------------------------------------------------------------------
                                                                                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                0.768    0.012 62.365 0.000  0.744  0.792
IT_diff_binned_sq[T.Interval(-0.47, -0.26, closed='right')]                             -0.032    0.010 -3.208 0.001 -0.051 -0.012
IT_diff_binned_sq[T.Interval(-0.26, 0.05, closed='right')]                              -0.017    0.009 -1.850 0.064 -0.035  0.001
IT_diff_binned_sq[T.Interval(0.05, 0.48, closed='right')]                               -0.077    0.010 -8.045 0.000 -0.095 -0.058
IT_diff_binned_sq[T.Interval(0.48, 1.0, closed='right')]                                -0.056    0.010 -5.614 0.000 -0.076 -0.037
retrocue_reliability[T.low]                                                             -0.027    0.009 -2.827 0.005 -0.045 -0.008
IT_diff_binned_sq[T.Interval(-0.47, -0.26, closed='right')]:retrocue_reliability[T.low]  0.017    0.013  1.268 0.205 -0.009  0.043
IT_diff_binned_sq[T.Interval(-0.26, 0.05, closed='right')]:retrocue_reliability[T.low]   0.000    0.014  0.027 0.978 -0.026  0.027
IT_diff_binned_sq[T.Interval(0.05, 0.48, closed='right')]:retrocue_reliability[T.low]    0.007    0.013  0.530 0.596 -0.019  0.033
IT_diff_binned_sq[T.Interval(0.48, 1.0, closed='right')]:retrocue_reliability[T.low]     0.032    0.013  2.410 0.016  0.006  0.059
Group Var                                                                                0.014    0.004                           
==================================================================================================================================

"""

In [35]:
md = smf.mixedlm("resp_correct ~ IT_diff_binned_sq*tested_item*retrocue_reliability", df, 
                 groups=df["participant"]).fit()
md.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                                            Mixed Linear Model Regression Results
=============================================================================================================================================================
Model:                                             MixedLM                                  Dependent Variable:                                  resp_correct
No. Observations:                                  41700                                    Method:                                              REML        
No. Groups:                                        139                                      Scale:                                               0.1836      
Min. group size:                                   300                                      Log-Likelihood:                                      -24115.9812 
Max. group size:                                   300                                      Converged:                                           Yes         
Mean group size:                                   300.0                                                                                                     
-------------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                                                                   Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                           0.701    0.028 25.392 0.000  0.647  0.756
IT_diff_binned_sq[T.Interval(-0.47, -0.26, closed='right')]                                                        -0.069    0.029 -2.380 0.017 -0.126 -0.012
IT_diff_binned_sq[T.Interval(-0.26, 0.05, closed='right')]                                                         -0.020    0.030 -0.687 0.492 -0.079  0.038
IT_diff_binned_sq[T.Interval(0.05, 0.48, closed='right')]                                                          -0.120    0.030 -3.951 0.000 -0.180 -0.061
IT_diff_binned_sq[T.Interval(0.48, 1.0, closed='right')]                                                           -0.074    0.030 -2.437 0.015 -0.134 -0.014
tested_item[T.prioritized]                                                                                          0.072    0.027  2.699 0.007  0.020  0.124
retrocue_reliability[T.low]                                                                                         0.028    0.030  0.923 0.356 -0.032  0.088
IT_diff_binned_sq[T.Interval(-0.47, -0.26, closed='right')]:tested_item[T.prioritized]                              0.067    0.031  2.168 0.030  0.006  0.128
IT_diff_binned_sq[T.Interval(-0.26, 0.05, closed='right')]:tested_item[T.prioritized]                               0.012    0.031  0.369 0.712 -0.050  0.073
IT_diff_binned_sq[T.Interval(0.05, 0.48, closed='right')]:tested_item[T.prioritized]                                0.059    0.032  1.828 0.067 -0.004  0.121
IT_diff_binned_sq[T.Interval(0.48, 1.0, closed='right')]:tested_item[T.prioritized]                                 0.033    0.032  1.013 0.311 -0.031  0.096
IT_diff_binned_sq[T.Interval(-0.47, -0.26, closed='right')]:retrocue_reliability[T.low]                             0.066    0.035  1.891 0.059 -0.002  0.133
IT_diff_binned_sq[T.Interval(-0.26, 0.05, closed='right')]:retrocue_reliability[T.low]                             -0.064    0.037 -1.733 0.083 -0.136  0.008
IT_diff_binned_sq[T.Interval(0.05, 0.48, closed='right')]:retrocue_reliability[T.low]                               0.090    0.036  2.500 0.012  0.020  0.161
IT_diff_binned_sq[T.Interval(0.48, 1.0, closed='right')]:retr

In [52]:
# md = smf.mixedlm("resp_correct ~ it_sim_dis_diff*reliability*validity + v2_sim_dis_diff*reliability*validity", df, 
#                  groups=df["participant"]).fit()
# md.summary()

In [51]:
# md = smf.mixedlm("resp_correct ~ it_sim_dis_diff*retrocue_reliability*tested_item + v2_sim_dis_diff*retrocue_reliability*tested_item", df, 
#                  groups=df["participant"]).fit()
# md.summary()

In [50]:
# md = smf.mixedlm("resp_correct ~ it_sim_dis_attend*reliability*validity + v2_sim_dis_attend*reliability*validity", df, 
#                  groups=df["participant"]).fit()
# md.summary()

In [49]:
# md = smf.mixedlm("resp_correct ~ it_sim_dis_unattend*reliability*validity + v2_sim_dis_unattend*reliability*validity", df, 
#                  groups=df["participant"]).fit()
# md.summary()

In [48]:
# md = smf.mixedlm("resp_correct ~ it_sim_dis_attend*reliability*validity + v2_sim_dis_diff*reliability*validity", df, 
#                  groups=df["participant"]).fit()
# md.summary()

In [47]:
# md = smf.mixedlm("resp_correct ~ it_sim_dis_unattend*reliability*validity + v2_sim_dis_diff*reliability*validity", df, 
#                  groups=df["participant"]).fit()
# md.summary()

In [42]:
df['validity']

0          valid
1          valid
2        invalid
3          valid
4        invalid
          ...   
41695      valid
41696      valid
41697      valid
41698      valid
41699      valid
Name: validity, Length: 41700, dtype: object

In [43]:
df['tested_item']

0          prioritized
1          prioritized
2        deprioritized
3          prioritized
4        deprioritized
             ...      
41695      prioritized
41696      prioritized
41697      prioritized
41698      prioritized
41699      prioritized
Name: tested_item, Length: 41700, dtype: object

In [44]:
df['retrocue_reliability']

0        high
1        high
2         low
3         low
4         low
         ... 
41695     low
41696     low
41697    high
41698    high
41699     low
Name: retrocue_reliability, Length: 41700, dtype: object

In [45]:
df['reliability']

0        0.833333
1        0.833333
2        0.666667
3        0.666667
4        0.666667
           ...   
41695    0.666667
41696    0.666667
41697    0.833333
41698    0.833333
41699    0.666667
Name: reliability, Length: 41700, dtype: float64

In [46]:

    
# md = smf.mixedlm("resp_correct ~ (it_sim_dis_diff + it_sim_dis_diff_sq) * reliability * validity_num + \
#   (v2_sim_dis_diff + v2_sim_dis_diff_sq) * reliability * validity_num", df, 
#                  groups=df["participant"]).fit()
# md.summary()